# Profiling and discussion

## Lorenz 96

In [35]:
# Add profiling code here
import numpy as np

def lorenz96(initial_state, nsteps, constants=(1/101, 100, 8)):
    """
    Perform iterations of the Lorenz 96 update.

    Parameters
    ----------
    initial_state : array_like or list
        Initial state of lattice in an array of floats.
    nsteps : int
        Number of steps of Lorenz 96 to perform.

    Returns
    -------
    numpy.ndarray
        Final state of lattice in an array of floats
    """

    alpha, beta, gamma = constants
    state = np.array(initial_state, dtype=float)
    N = len(state)
    new_state = np.empty_like(state)  # Create a new state array

    for _ in range(nsteps):
        # Compute the first two elements
        new_state[0] = alpha * (beta * state[0] + (state[N - 2] - state[1]) * state[N - 1] + gamma)
        new_state[1] = alpha * (beta * state[1] + (state[N - 1] - state[0]) * state[0] + gamma)

        # Compute the elements between 2 and N-2
        new_state[2:N - 2] = alpha * (beta * state[2:N - 2] + (state[0:N - 4] - state[3:N - 1]) * state[1:N - 3] + gamma)   

        # Compute the last element
        new_state[N - 1] = alpha * (beta * state[N - 1] + (state[N - 3] - state[0]) * state[N - 2] + gamma)

        # Update the state array
        state[:] = new_state

    return state


initial_state = np.full(49, 8.0)
initial_state = np.insert(initial_state,2,9.0)
nsteps = 50

In [36]:
%timeit lorenz96(initial_state, nsteps)

1.03 ms ± 64.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### Add discussion here

## First Version
By interacting with chatgpt I realised that explcitly stating the data type allows numpy to optimise operations pertaining to the specific data. After this i used the timeit to profile my code and got a speed of 166 microseconds.
## Second Version
Intead of using nested loops i decided to use numpys abilties to "vectorize" the function in order to optimise.After vecotrising i found that the code ran slower for smaller steps(20) but faster than my first version for larger steps(50)
## Third Version
After consulting chatgpt it seems the action of change a new array is inneficient due to memory allocation and deallocation. This didnt seem to make the program as efficient.
## Fourth Version
Now that ive applied numpys capabilities where i can to optimise. I decided to try an optimise the actual algorithm. Instead of using one algorithm for every part of the array the calculation was split to the first, second, last and all the arrays in between. This allowed a much faster vecotrisation process.

Compared to my original code I have managed to optimise my code to go from 1.03 milliseconds to 228 microseconds for the same array.. I have changed the algorithm and used vectorisation where i could to make it more efficient.

## Game of Life

In [ ]:
# Add profiling code here


#### Add discussion here

Replace me.